# Notebooks pour Allgo
[Allgo](https://allgo.inria.fr/) est un service qui offre une interface web pour des outils en ligne de commande. Il dispense ainsi les utilisateurs d'installer les programmes en question et offre la possibilité de cacher leur code source et modèles (pour les outils ayant besoin d'un entraînement). Il permet donc : 
* D'évaluer des outils de recherche (par des groupes de recherche et des industriels ?)
* D'archiver un environnement d'exécution

Ce carnet est la suite du [premier](step_1.ipynb); ici, on se propose de partir de zéro et de faire toutes les requêtes à Allgo depuis notre carnet. 

## Speads

[Speads](https://allgo.inria.fr/app/speads) est un exemple d'application disponible sur Allgo. Speads segmente une conversation, identifie les locuteurs et tente de déterminer leur genre.

Par exemple, en partant de cette conversation :

In [ ]:
from IPython.display import Audio
Audio("conv1.mp3")

On obtient en sortie le tableau suivant :

In [ ]:
import csv
with open('conv1_speads.tsv') as tsvfile:
    reader = csv.reader(tsvfile, delimiter='\t')
    for row in reader:
        print(row)

Plutôt qu'un simple tableau, on voudrait obtenir un graphique de ce type:
![graphique](plot_1.png)

Ici par exemple, on voit bien plus facilement qu'on a cinq locuteurs dont deux sont des hommes.

Pour obtenir ce graphique, *et pour d'autres tâches d'analyse* -- il ne s'agit pas que de visualisation--, on a besoin d'un environnement plus puissant que l'UI d'Allgo, ce qui justifie le recours à un notebook type Jupyter. Par exemple, dans les données brutes, le genre du locuteur est mélangé avec son identifiant (comme dans `speaker4_F`), ce qui est problématique.

## Récupérer les données via l'API REST

Dans le [notebook précédent](step_1.ipynb), on était parti de données récupérées à la main sur Allgo.

Ici, on part simplement du fichier audio de conversation, et on veut interroger allgo directement (pour pouvoir facilement ajuster des paramètres, par exemple).

Allgo s'interroge comme suit:
* Le client `POST`e une requête (ID_application, paramètres, fichier(s) d'entrée)
* Réponse contenant un identifiant
* ... le client interroge le serveur pour suivre le job ...
* Réponse contenant une liste de fichiers téléchargeables
* Le client télécharge le ou les fichiers qui l'intéressent


In [6]:
# La librairie requests est nécessaire (pip install requests)
import requests
TOKEN = "***REMOVED***" # privé, peut être regénéré
APP_ID = 99

def start_job(token, app_id, files, params):
    """Launches job, returns job ID"""
    headers = {'Authorization': 'Token token={}'.format(token)}
    requests.post('https://allgo.inria.fr/api/v1/jobs', headers=headers)
    pass

def check_job(id):
    pass


## Préparer les données

Ici, on corrige le problème évoqué plus haut (genre et identifiant du locuteur mélangés)

In [ ]:
import csv
def preprocess_raw_speads_output(infile, outfile):
    with open(outfile, 'w') as dest:
        dest.write("ID\tGender\tStart\tEnd\n")
        with open(infile, 'r') as source:
            reader = csv.reader(source, delimiter='\t')
            for row in reader:
                dest.write('{}\t{}\t{}\t{}\n'.format(row[0][:-2], row[0][-1:], row[1], row[2]))

In [ ]:
preprocess_raw_speads_output(infile='conv1_speads.tsv', outfile='out/conv1_processed.tsv')

Jetons un oeil aux données après traitement :

In [6]:
import pandas as pd
data = pd.read_csv('out/conv1_processed.tsv', sep='\t')
data


,ID,Gender,Start,End
0,speaker0,F,0.00000,1.09000
1,speaker0,F,1.65000,2.60000
2,speaker1,M,3.07000,4.56000
3,speaker2,M,7.35000,20.74000
4,speaker0,F,20.74000,23.20000
5,speaker3,F,23.20000,24.73000
6,speaker0,F,24.73000,27.33000
7,speaker4,F,29.98000,41.44000
8,speaker3,F,41.44000,43.91000
9,speaker1,M,46.53000,75.32000


## Visualiser la conversation avec Vega-Lite

In [ ]:
import vega
vega.VegaLite({
  "$schema": "https://vega.github.io/schema/vega-lite/v2.json",
  "description": "Conversation timeline",
  "mark": "bar",
  "encoding": {
    "y": {"field": "ID", "type": "nominal"},
    "x": {"field": "Start", "type": "quantitative"},
    "x2": {"field": "End", "type": "quantitative"},
    "color": {"field": "Gender", 
              "type": "nominal",
              "scale": {
                "domain": ["F","M"],
                 "range": ["#ff99ff","#4169e1"]
      }
    }
  }
},
data)

... ce qui n'est pas le résultat attendu.

* Première surprise : pas de support stable de Vega-Lite dans JupyterLab (alors qu'il s'agit d'un plugin *core*). Au 3 octobre 2017, la bibliothèque python nécessaire n'est pas disponible par `pip` (et un build manuel échoue).
  - Donc, on ne peut embarquer la visualisation précédente que dans Jupyter Classic, à moins d'utiliser des appels bas niveau ? 
* Deuxième surprise : dans Jupyter Classic, le plugin Vega-Lite ne permet pas les visualisations superposées (*layered*). J'ai ouvert [un ticket](https://github.com/altair-viz/jupyter_vega/issues/38) à ce sujet.
* Apparemment, la version de vega-lite utilisée dans le plugin *core* est la v1 (la version actuelle est la v2).

On peut contourner ce problème avec une approche bas niveau (qu'on aimerait quand même éviter...)

In [10]:
def display_static_conversation(dataframe):
  from IPython.display import display
  import json
  bundle_contents = {
  "$schema": "https://vega.github.io/schema/vega-lite/v2.json",
  "description": "Conversation timeline",
  "data": {
    "values": json.loads(dataframe.to_json(orient='records'))
  },
  "mark": "bar",
  "encoding": {
    "y": {"field": "ID", "type": "nominal"},
    "x": {"field": "Start", "type": "quantitative"},
    "x2": {"field": "End", "type": "quantitative"},
    "color": {"field": "Gender", 
              "type": "nominal",
              "scale": {
                "domain": ["F","M"],
                 "range": ["#ff99ff","#4169e1"]
              }
          }
      }
  }
  mime_bundle = {'application/vnd.vegalite.v1+json': bundle_contents}
  display(mime_bundle, raw=True)

display_static_conversation(data)

(on note dans le type MIME que vegalite a une version de retard...)



## Discussion



## Annexe: nettoyage

In [ ]:
def clean():
    import os
    os.system('rm out/*')



In [ ]:
clean()